# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [27]:
#Loading data set
import pandas as pd 
data = pd.read_csv('homepage_actions.csv')
print (data.head())

                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view


In [28]:
print(data.dtypes)

timestamp    object
id            int64
group        object
action       object
dtype: object


In [29]:
print(data['id'].nunique())

6328


In [30]:
print(data.loc[data['action'] == 'click', 'id'].nunique())

1860


In [31]:
print(data.loc[(data['action'] == 'click') & (data['action'] == 0), 'id'].nunique())

0


In [32]:
# Create a new column to store the group a user belongs to
data['group'] = data['group'].apply(lambda x: 'control' if x == 0 else 'experimental')

# Calculate the number of unique users in each group
print(data['group'].value_counts())

experimental    8188
Name: group, dtype: int64


In [33]:
print(data.loc[data['group'] == 'control', 'id'].isin(data.loc[data['group'] == 'experimental', 'id']).value_counts())

Series([], Name: id, dtype: int64)


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [34]:
from scipy import stats
# Extract the counts of actions for the control group and the experimental group
control_actions = data.loc[data['group'] == 'control', 'action'].value_counts()
experimental_actions = data.loc[data['group'] == 'experimental', 'action'].value_counts()

# Combine the counts into a single DataFrame
counts_data = pd.DataFrame({'control': control_actions, 'experimental': experimental_actions})

# Perform the Chi-Square Test for Independence
chi2, p_value, _, _ = stats.chi2_contingency(counts_data)

print("Chi-Square statistic: ", chi2)
print("P-value: ", p_value)

Chi-Square statistic:  nan
P-value:  nan


c:\Users\Branton\anaconda2\envs\learn-env\lib\site-packages\scipy\stats\contingency.py:242: RuntimeWarning: invalid value encountered in less
  if np.any(observed < 0):
c:\Users\Branton\anaconda2\envs\learn-env\lib\site-packages\scipy\stats\contingency.py:267: RuntimeWarning: invalid value encountered in sign
  observed = observed + 0.5 * np.sign(expected - observed)
c:\Users\Branton\anaconda2\envs\learn-env\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [35]:
import scipy.stats as stats

# Assuming you have the following data
control_group_ctr = 0.05  # Example control group click-through rate
experiment_group_clicks = 300  # Example actual clicks in the experiment group
experiment_group_impressions = 5000  # Example total impressions in the experiment group

# Step 1: Calculate the Expected Number of Clicks
expected_clicks = control_group_ctr * experiment_group_impressions

# Step 2: Calculate the Number of Standard Deviations (z-score)
z_score = (experiment_group_clicks - expected_clicks) / (expected_clicks ** 0.5)

# Step 3: Calculate a p-value using the Normal Distribution
p_value = 2 * (1 - stats.norm.cdf(abs(z_score)))  # Multiply by 2 for a two-tailed test

# Display the results
print(f"Expected Clicks: {expected_clicks:.2f}")
print(f"Z-Score: {z_score:.2f}")
print(f"P-Value: {p_value:.4f}")


Expected Clicks: 250.00
Z-Score: 3.16
P-Value: 0.0016


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [36]:
# Assuming you have the following data
experiment_group_clicks = 300  # Example actual clicks in the experiment group

# Calculate the z-score
z_score = (experiment_group_clicks - expected_clicks) / (expected_clicks ** 0.5)

# Display the result
print(f"Z-Score: {z_score:.2f}")


Z-Score: 3.16


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [37]:
import scipy.stats as stats

# Calculate a p-value using the Normal Distribution
p_value = 2 * (1 - stats.norm.cdf(abs(z_score)))  # Multiply by 2 for a two-tailed test

# Display the result
print(f"P-Value: {p_value:.4f}")


P-Value: 0.0016


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Yes it does**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.